In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Initilisation / upload of librairies
!pip install import_ipynb
!pip install panda
import os
import import_ipynb
import pandas as pd
import numpy as np
from datetime import datetime
import os

class style():
  BLACK = '\033[30m'
  RED = '\033[31m'
  GREEN = '\033[32m'
  YELLOW = '\033[33m'
  BLUE = '\033[34m'
  MAGENTA = '\033[35m'
  CYAN = '\033[36m'
  WHITE = '\033[37m'
  UNDERLINE = '\033[4m'
  RESET = '\033[0m'

%cd '/content/drive/MyDrive/Wild Code School/Kung Fu Pandas/Modules/'
from data_formatting import *

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=fdb355861a324dce2132b224b2b26b401e4302c141e6a35ba31c900390f4ff94
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb
  Created wheel for panda: filename=panda-0.3.1-py3-none-any.whl size=7255 sha256=1f68e2b8396fc036607eb4957d541b9f23c9a087db1b1e962e4b6601baecb223
  Stored in directory: /root/.cache/pip/wheels/e6/3d/81/a3665ce657d35359ca337b1db2975dbe5cd281a88b8982f6b6
Successfully built panda
/content/drive/.shortcut-targets-by-id/1iYrLEW3yrtr1S1PEjB-TFMG9lQLdWpC1/Kung Fu Pandas/Modules
importing Jupyter notebook from data_formatting.ipynb


In [ ]:
#import data tables from 'movies data repertory'
file_path = '/content/drive/MyDrive/Wild Code School/Kung Fu Pandas/Database/Movies_data/'
separator = "\t"
file_names = {'titl2_df':'title.basics.tsv.gz',
              'titl1_df':'title.akas.tsv.gz',
              'crew_df':'title.crew.tsv.gz',
              'principals_df' :'title.principals.tsv.gz',
              'rating_df':'title.ratings.tsv.gz',
              'name_df':'name.basics.tsv.gz'}
          
df_dict = {'titl2_df':'','titl1_df':'','crew_df':'','principals_df':'','rating_df':'','name_df':''}
columns_names = {'titl2_df':['titleID','titletype','primarytitle','originaltitle','isadult','startyear','endyear','runtimeminutes','genre'],
                 'titl1_df':['titleID','ordering','title','region','country','language','types','attributes','isoriginaltitle'],
                 'crew_df' :['titleID','directors','writers'],
                 'principals_df':['titleID','ordering','nameID','category','job','characters'],
                 'rating_df':['titleID','averagerating','numvotes'],
                 'name_df':['nameID','name','birthyear','deathyear','primaryprofession','mainmovies']}

for keys,values in df_dict.items():
  df_dict.update({keys : import_database(file_path,file_names[keys],separator,columns_names[keys])})

In [ ]:
#-------------------------------------------------
#NEW CODE FOR IMPORT TOP RATING MOVIES "COUNTRIES"
#-------------------------------------------------

#Import countries(ImdB scraping)
file_path = '/content/drive/MyDrive/Wild Code School/Kung Fu Pandas/Database/Imdb scrapping/'
separator = "\t"
file_name = 'countries.tsv.gz'   
columns_names = ['titleID','title','country']
df_countries_imdb = import_database(file_path,file_name,separator,columns_names)


In [ ]:
#-------------------------------------------------
#NEW CODE FOR IMPORT URL / SYNOPSIS
#-------------------------------------------------

#import url & synopsis (ImdB scrappin Antony)
file_path = '/content/drive/MyDrive/Wild Code School/Kung Fu Pandas/Database/Imdb scrapping/'
separator = "\t"
file_name = 'synopsis_poster.csv'  
columns_names = ['titleID','title','country']
df_synopsis_poster_imdb = pd.read_csv(file_path+file_name,sep=',')
df_synopsis_poster_imdb = df_synopsis_poster_imdb.drop(columns='Unnamed: 0')

In [ ]:
df_countries_imdb = df_countries_imdb.loc[:,['titleID','country']]
df_countries_imdb['country'] = df_countries_imdb.loc[:,'country'].apply(lambda x: x.strip("[]' "))
df_countries_imdb['country'] = df_countries_imdb['country'].apply(lambda x: x.replace("'",""))

In [ ]:
#Choice 2 : filtering titl1 data
df_dict['titl1_df'] = df_dict['titl1_df'][df_dict['titl1_df']['isoriginaltitle'] == 1]
df_dict['titl1_df'].drop_duplicates(subset='titleID',inplace=True,keep='last')

In [ ]:
#Suppression of unecessary columns (not to be used by the ML model)
df_dict['titl2_df'] = df_dict['titl2_df'].drop(columns=['titletype','primarytitle','originaltitle','endyear']) 
df_dict['titl1_df'] = df_dict['titl1_df'].drop(columns=['ordering','region','language','types','attributes','isoriginaltitle','country'])
df_dict['principals_df'] = df_dict['principals_df'].drop(columns=['ordering','job','characters'])
df_dict['name_df'] = df_dict['name_df'].drop(columns=['birthyear','deathyear','primaryprofession','mainmovies'])

In [ ]:
df_directors = df_dict['principals_df'][df_dict['principals_df']['category'] == 'director']

In [ ]:
df_directors = pd.merge(df_directors,df_countries_imdb,on='titleID',how='inner')
df_directors.drop_duplicates(subset='titleID',inplace=True,keep='first')
df_directors.drop(columns='country',inplace=True)
df_directors = pd.merge(df_directors,df_dict['name_df'],how='left',on='nameID')
df_directors.drop(columns=['category','nameID'],inplace=True)
df_directors.rename(columns={'name' : 'director'},inplace=True)

In [ ]:
df_actors = df_dict['principals_df'][df_dict['principals_df']['category'] == 'actor']
df_actress = df_dict['principals_df'][df_dict['principals_df']['category'] == 'actress']

In [ ]:
df_act = pd.concat([df_actors,df_actress])
df_act = pd.merge(df_countries_imdb,df_act,how='left',on='titleID')

In [ ]:
df_act = pd.merge(df_act,df_dict['name_df'],how='left',on='nameID')

In [ ]:
df_act.head(5)

,titleID,country,nameID,category,name
0,tt0006864,United States,nm0366008,actor,Robert Harron
1,tt0006864,United States,nm0877548,actor,F.A. Turner
2,tt0006864,United States,nm0001273,actress,Lillian Gish
3,tt0006864,United States,nm0550615,actress,Mae Marsh
4,tt0010323,Germany,nm0470328,actor,Werner Krauss


In [ ]:
df_act.drop(columns=['country','category','nameID'],inplace=True)

In [ ]:
df_act = df_act.astype(str)
nb_max_actors_per_movie = df_act['titleID'].value_counts().max()
print(nb_max_actors_per_movie)
# 10 actors max per movie

10


In [ ]:
df_act = df_act.groupby('titleID')['name'].apply(','.join).reset_index()

In [ ]:
df_act.head(1)

,titleID,name
0,tt0006864,"Robert Harron,F.A. Turner,Lillian Gish,Mae Marsh"


In [ ]:
df_act = df_act['name'].str.split(',',expand=True,)

In [ ]:
df_act = df_act.fillna('unknown')

In [ ]:
df_act

,0,1,2,3,4,5,6,7,8,9
0,Robert Harron,F.A. Turner,Lillian Gish,Mae Marsh,unknown,unknown,unknown,unknown,unknown,unknown
1,Werner Krauss,Conrad Veidt,Friedrich Feher,Lil Dagover,unknown,unknown,unknown,unknown,unknown,unknown
2,B.F. Blinn,Charles Chaplin,Jackie Coogan,Carl Miller,Albert Austin,Henry Bergman,Edward Biby,Edna Purviance,Beulah Bains,Nellie Bly Baker
3,Einar Axelsson,Victor Sjöström,Tore Svennberg,Tor Weijden,Hilda Borgström,Astrid Holm,Concordia Selander,Lisa Lundholm,unknown,unknown
4,Benjamin Christensen,Elisabeth Christensen,Maren Pedersen,Clara Pontoppidan,unknown,unknown,unknown,unknown,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...
5455,David Strathairn,Gay DeForest,Frances McDormand,Linda May,unknown,unknown,unknown,unknown,unknown,unknown
5456,Chris Pratt,J.K. Simmons,Yvonne Strahovski,Betty Gilpin,unknown,unknown,unknown,unknown,unknown,unknown
5457,Daniel Kaluuya,LaKeith Stanfield,Jesse Plemons,Dominique Fishback,unknown,unknown,unknown,unknown,unknown,unknown
5458,Hakim Faris,Patrick d'Assumçao,Alfonso Arfi,Victoire Du Bois,unknown,unknown,unknown,unknown,unknown,unknown


In [ ]:
#Merge tables
merged_df = pd.merge(df_dict['titl1_df'],df_dict['titl2_df'],how='left',on='titleID')
merged_df = pd.merge(merged_df,df_dict['rating_df'],how='left',on='titleID')
merged_df = pd.merge(merged_df,df_directors,how='left',on='titleID')

In [ ]:
merged_df

,titleID,title,isadult,startyear,runtimeminutes,genre,averagerating,numvotes,director
0,tt0000502,Bohemios,0.0,1905.0,100.0,NaN,4.0,14.0,NaN
1,tt0000574,The Story of the Kelly Gang,0.0,1906.0,70.0,"Action,Adventure,Biography",6.0,699.0,NaN
2,tt0000591,L'enfant prodigue,0.0,1907.0,90.0,Drama,5.0,16.0,NaN
3,tt0000615,Robbery Under Arms,0.0,1907.0,NaN,Drama,4.0,23.0,NaN
4,tt0000630,Amleto,0.0,1908.0,NaN,Drama,4.0,23.0,NaN
...,...,...,...,...,...,...,...,...,...
420442,tt9916362,Akelarre,0.0,2020.0,92.0,"Drama,History",6.0,3998.0,NaN
420443,tt9916428,Hong xing zhao yao Zhong guo,0.0,2019.0,NaN,"Adventure,History,War",4.0,14.0,NaN
420444,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,0.0,2015.0,57.0,Documentary,NaN,NaN,NaN
420445,tt9916680,De la ilusión al desconcierto: cine colombiano...,0.0,2007.0,100.0,Documentary,NaN,NaN,NaN


In [ ]:
#Getting movies countries from dataframe df_countries_imdb (already filtered with nb votes and average rating)
merged_df = pd.merge(df_countries_imdb,merged_df,how='left',on='titleID')
#Getting movies synopsis and url  from dataframe df_synopsis_poster_imdb (already filtered nb votes and average rating)
merged_df = pd.merge(merged_df,df_synopsis_poster_imdb,how='left',left_on='titleID',right_on='imdb_id')


In [ ]:
merged_df.head(1)

,titleID,country,title,isadult,startyear,runtimeminutes,genre,averagerating,numvotes,director,imdb_id,synopsis,poster_url
0,tt0006864,United States,Intolerance: Love's Struggle Throughout the Ages,0.0,1916.0,163.0,"Drama,History",8.0,14992.0,D.W. Griffith,tt0006864,"The story of a poor young woman, separated by ...",https://www.themoviedb.org/t/p/original/mATFEp...


In [ ]:
#Suppression of IDs (no more merge needed)
merged_df.drop(columns=['imdb_id'],inplace=True)

In [ ]:
#saving merged_df as ML_DataFrame.csv
file_path = '/content/drive/MyDrive/Wild Code School/Kung Fu Pandas/Database/Machine learning Dataset/'
file_name = 'ML_dataFrame.csv'
merged_df.to_csv(file_path+file_name,sep=',',index=False)

In [ ]:
#Filling NaN

In [ ]:
merged_df.isna().sum()

titleID             0
country             0
title               0
isadult             0
startyear           0
runtimeminutes      0
genre               0
averagerating       0
numvotes            0
director          262
synopsis            1
poster_url          2
dtype: int64

In [ ]:
merged_df.fillna('unknown',inplace=True)

In [ ]:
merged_df.shape

(5460, 12)

In [ ]:
#Get dummies for 'genre' & 'countries' & Directors
genre_df = pd.get_dummies(merged_df['genre'].str.split(',',expand=True).replace({None:np.nan}).stack()).groupby(level=0).max()
countries_df = pd.get_dummies(merged_df['country'].str.split(',',expand=True).replace({None:np.nan}).stack()).groupby(level=0).max()
directors_df = pd.get_dummies(merged_df['director'].str.split(',',expand=True).replace({None:np.nan}).stack()).groupby(level=0).max()
ML_model_df = pd.merge(merged_df,genre_df,left_index=True,right_index=True)
ML_model_df = pd.merge(ML_model_df,countries_df,left_index=True,right_index=True)
ML_model_df = pd.merge(ML_model_df,directors_df,left_index=True,right_index=True)

In [ ]:
ML_model_df.shape

(5460, 2571)

In [ ]:
#Define Model
from sklearn.neighbors import NearestNeighbors
y = ML_model_df[['title']]
#X = ML_model_df.drop(columns=['titleID','title','isadult','runtimeminutes','genre','averagerating','numvotes','country','synopsis','poster_url','director'])
X = ML_model_df.iloc[:,12:]
n_neighbors = 10
model = NearestNeighbors(n_neighbors=n_neighbors)
model.fit(X)

NearestNeighbors(n_neighbors=10)

In [ ]:
###################################
########  USER INTERFACE ##########
###################################

check = 'nok'
while check == 'nok' :
  print(style.BLUE,'\033[34m',"\nEnter movie key word",style.WHITE,)                 
  title = input('')
  list_movies = []
  list_index = ML_model_df['title'].index[ML_model_df['title'].str.contains(title)].to_list()
  if len(list_index) == 0 :  #no match
    print(style.BLUE,'\033[34m',"\nNo movie matching your request, please enter new key words",style.WHITE)
  else :
    for index in list_index :
      list_movies.append(ML_model_df.loc[index,'title'])
    if len(list_movies) > 10:
      print(style.BLUE,'\033[34m',"\nToo many movies match with you input, please refine your research",style.WHITE)
    else :
      print(style.BLUE,'\033[34m','\nDoes one of these movie matches with your request?',style.WHITE)
      print(list_movies)
      print(style.BLUE,'\033[34m','\nPlease enter Y / N',style.WHITE)
      match = input((''))
      if match == 'N':
        print(style.BLUE,'\033[34m','\nPlease retry with other key words',style.WHITE)
      elif match =='Y':
        while check == 'nok' :
          print(style.BLUE,'\nPlease enter the full name : ',style.WHITE)
          title = input('')
          if (ML_model_df['title'] == title).sum() > 0 :
            #X_input = ML_model_df[ML_model_df['title'] == title].drop(columns=['titleID','title','isadult','runtimeminutes','genre','averagerating','numvotes','country','synopsis','poster_url','director'])
            X_input = ML_model_df[ML_model_df['title'] == title].iloc[:,12:]
            check = 'ok'
          else :
            print(style.BLUE,'\033[34m','\nWrong name, try again',style.WHITE)         
      
results = model.kneighbors(X_input)
merged_df.loc[[i for i in results[1][0,:]],['title','startyear','genre','director','runtimeminutes','isadult','averagerating','numvotes','synopsis','poster_url']]

  
Enter movie key word 
Django
  
Does one of these movie matches with your request? 
['Django', 'Django Unchained']
  
Please enter Y / N 
Y
 
Please enter the full name :  
Django Unchained


,title,startyear,genre,director,runtimeminutes,isadult,averagerating,numvotes,synopsis,poster_url
4264,Django Unchained,2012.0,"Drama,Western",Quentin Tarantino,165.0,0.0,8.0,1432814.0,"With the help of a German bounty hunter, a fre...",https://www.themoviedb.org/t/p/original/7oWY8V...
292,Johnny Guitar,1954.0,"Drama,Western",Nicholas Ray,110.0,0.0,8.0,16788.0,"On the outskirts of town, the hard-nosed Vienn...",https://www.themoviedb.org/t/p/original/slMEKs...
1215,Pale Rider,1985.0,"Drama,Western",unknown,115.0,0.0,7.0,56561.0,A mysterious preacher protects a humble prospe...,https://www.themoviedb.org/t/p/original/yUqCtt...
497,Hud,1963.0,"Drama,Western",Martin Ritt,112.0,0.0,8.0,20931.0,Hud Bannon is a ruthless young man who tarnish...,https://www.themoviedb.org/t/p/original/54HnZL...
334,Giant,1956.0,"Drama,Western",George Stevens,201.0,0.0,8.0,37152.0,Giant is a movie of huge scale and grandeur in...,https://www.themoviedb.org/t/p/original/alRcXV...
480,Ride the High Country,1962.0,"Drama,Western",Sam Peckinpah,94.0,0.0,8.0,12845.0,An ex-lawman is hired to transport gold from a...,https://www.themoviedb.org/t/p/original/oBdJli...
5426,First Cow,2019.0,"Drama,Western",Kelly Reichardt,122.0,0.0,7.0,15004.0,A taciturn loner and skilled cook has traveled...,https://www.themoviedb.org/t/p/original/yS41cr...
1788,Pulp Fiction,1994.0,"Crime,Drama",Quentin Tarantino,154.0,0.0,9.0,1915188.0,"A burger-loving hit man, his philosophical par...",https://www.themoviedb.org/t/p/original/d5iIlF...
5174,The Rider,2017.0,"Drama,Western",Chloé Zhao,104.0,0.0,8.0,17443.0,"Once a rising star of the rodeo circuit, and a...",https://www.themoviedb.org/t/p/original/cFsrA0...
602,Hang 'Em High,1968.0,"Drama,Western",Ted Post,114.0,0.0,7.0,37333.0,"Marshall Jed Cooper survives a hanging, vowing...",https://www.themoviedb.org/t/p/original/gFvRKg...


In [ ]:
results[1][0,:]

array([4264,  292, 1215,  497,  334,  480, 5426, 1788, 5174,  602])

In [ ]:
[i for i in results[1][0,:]]

[4264, 292, 1215, 497, 334, 480, 5426, 1788, 5174, 602]